# DREAMTEAM PROJECT 1

## Datasets to be used
    1. Trading cards (high liquidity cards) - 
    
    2. Bitcoin current price - https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD
    
    3. Bitcoin historical data - https://www.google.com/finance/quote/BTC-AUD
    
    4. Government bond (baseline) - https://www.google.com/finance/quote/GOVT:ASX?window=5Y
    
    5. ASX200 (general stock market) - https://www.google.com/finance/quote/XJO:INDEXASX
    
    6. Property - ABS data in 8 capital cities- https://www.abs.gov.au/statistics/economy/price-indexes-and-inflation/residential-property-price-indexes-eight-capital-cities/latest-release

### Why we chose these measures

    5. ASX 500: The S&P/ASX 200 index is a market-capitalization weighted and float-adjusted stock market index of stocks listed on the Australian Securities Exchange. The index is maintained by Standard & Poor's and is considered the benchmark for Australian equity performance.

In [139]:
# Intialising imports
import os
import requests
import pandas as pd
import numpy as np
import json
import datetime as dt
from pathlib import Path
import seaborn as sns

%matplotlib inline

###  DATA CLEAN

In [140]:
# Importing Bitcoin current price
# Bitcoin API URL
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=AUD"

# Fetch current BTC price
btc_response = requests.get(btc_url)
btc_content = btc_response.content
btc_data = btc_response.json()
#print (json.dumps(btc_data, indent=4))
btc_price_current = float (btc_data["data"]["1"]["quotes"]["USD"]["price"])
print (btc_price_current)

36978.0


In [141]:
# Importing Bitcoin historical data

# Set the file path
btc_path=Path("btc_historical.csv")

# Reading the data
btc_df=pd.read_csv(btc_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
btc_df.index = btc_df.index.date

# Ensuring data loaded correctly
btc_df.head()

,Close
2015-11-19,454.097675
2015-11-20,445.560711
2015-11-22,447.867676
2015-11-23,449.365998
2015-11-24,442.422744


In [142]:
# Cleaning Bitcoin historical data

# Count nulls
btc_df.isnull().mean()

# Drop nulls
btc_df = btc_df.dropna()

# Checking nulls are dropped
btc_df.isnull().sum()

Close    0
dtype: int64

In [143]:
# Importing and cleaning ASX200 historical data

# Set the file path
asx200_path=Path("asx200_historical.csv")

# Reading the data
asx200_df=pd.read_csv(asx200_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
asx200_df.index = asx200_df.index.date

# Ensuring data loaded correctly
asx200_df.head()

,Close
2011-05-20,4732.2
2011-05-23,4643.0
2011-05-24,4628.8
2011-05-25,4584.7
2011-05-26,4660.2


In [144]:
asx200_df.isnull().mean()

Close    0.0
dtype: float64

In [145]:
# Importing and cleaning bonds historical data

# Set the file path
bonds_path=Path("asx_bonds.csv")

# Reading the data
bonds_df=pd.read_csv(bonds_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
bonds_df.index = bonds_df.index.date

# Ensuring data loaded correctly
bonds_df.head()

,Close
2012-07-27,25.00
2012-08-08,24.57
2012-09-12,24.92
2012-09-19,24.78
2012-09-24,24.71


In [146]:
bonds_df.isnull().mean()

Close    0.0
dtype: float64

In [147]:
# Importing and cleaning property historical data

# Set the file path
property_path=Path("property_history.csv")

# Reading the data
property_df=pd.read_csv(property_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
#property_df.index = property_df.index.date

# Ensuring data loaded correctly
property_df.head()


,Weighted Average of eight Capital Cities
Date,
2003-09-01,69.0
2003-12-01,71.6
2004-03-01,71.3
2004-06-01,70.6
2004-09-01,70.6


In [148]:
property_df.dropna()
property_df.isnull().mean()

Weighted Average of eight Capital Cities    0.0
dtype: float64

In [149]:
# All Nulls removed from data sets

print (f"BTC is {btc_df.isnull().sum()}")
print (f"asx200 is {asx200_df.isnull().sum()}")
print (f"bonds is {bonds_df.isnull().sum()}")
print (f"property is {property_df.isnull().sum()}")

BTC is Close    0
dtype: int64
asx200 is Close    0
dtype: int64
bonds is Close    0
dtype: int64
property is Weighted Average of eight Capital Cities    0
dtype: int64


# ANALYSIS 

## TRADITIONAL PORTFOLIO
    1. ASX200
    2. Bonds
    3. Property

In [150]:
# Checking data types and setting them to float variables as needed
print (asx200_df['Close'].dtype)
print (bonds_df['Close'].dtype)
print (property_df['Weighted Average of eight Capital Cities'].dtype)

# All data types are currently float variables

float64
float64
float64


In [151]:
# Calculating daily returns for ASX 200
asx200_df_daily_returns = asx200_df.pct_change()
asx200_df_daily_returns.columns =["ASX 200"]
asx200_df_daily_returns.head()

,ASX 200
2011-05-20,NaN
2011-05-23,-0.018850
2011-05-24,-0.003058
2011-05-25,-0.009527
2011-05-26,0.016468


In [152]:
# Calculating daily returns for Bonds
bonds_df_daily_returns = bonds_df.pct_change()
bonds_df_daily_returns.columns =["Bonds"]
bonds_df_daily_returns.head()

,Bonds
2012-07-27,NaN
2012-08-08,-0.017200
2012-09-12,0.014245
2012-09-19,-0.005618
2012-09-24,-0.002825


In [156]:
# Calculating quarterly returns for Weighted Average
property_df_quarterly_returns = property_df.pct_change()
property_df_quarterly_returns.head()

,Weighted Average of eight Capital Cities
Date,
2003-09-01,NaN
2003-12-01,0.037681
2004-03-01,-0.004190
2004-06-01,-0.009818
2004-09-01,0.000000


### Traditional portfolio - Quarterly Returns

In [158]:
# Combining datasets to create traditional portfolio
traditional_df_quarterly_returns = pd.concat([asx200_df_daily_returns, bonds_df_daily_returns, property_df_quarterly_returns], axis="columns", join="outer")
traditional_df_quarterly_returns.dropna().head()

,ASX 200,Bonds,Weighted Average of eight Capital Cities
2015-06-01,-0.007235,0.001923,0.046683
2015-09-01,-0.021241,0.006551,0.020344
2016-03-01,0.008482,0.001526,-0.001531
2016-06-01,-0.010300,-0.001139,0.019939
2016-09-01,-0.003203,0.004104,0.015038


### Traditional portfolio - Daily Returns

In [133]:
traditional_df_daily_returns = pd.concat([asx200_df_daily_returns, bonds_df_daily_returns], axis="columns", join="inner")
traditional_df_daily_returns.dropna().head()

,ASX 200,Bonds
2012-08-08,0.004893,-0.017200
2012-09-12,0.008207,0.014245
2012-09-19,0.005393,-0.005618
2012-09-24,-0.005172,-0.002825
2012-10-08,-0.002781,0.011736
